Loading Libraries

In [30]:
import numba
import tensorflow as tf
import pandas as pd
import numpy as np
import random
import gc  # garbage collector

from fastcore.basics import *
from fastcore.parallel import *
from numba import jit, njit, vectorize, cuda, uint32, f8, uint8

from sklearn.model_selection import train_test_split
from sklearn import metrics  # for accuracy calculation
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from functools import partial
from os import cpu_count
import matplotlib.pyplot as plt

import math
from pylab import imshow, show
from timeit import default_timer as timer

from dask import dataframe as dd
from dask.distributed import Client

Setting up paths to csv files / datasets

In [31]:
# CSV-01-12
path_DdoS_DNS = "C:\\CIC-DDOS-2019\\DrDoS_DNS.csv"
path_DdoS_MSSQL = "C:\\CIC-DDOS-2019\\DrDoS_MSSQL.csv"
path_DdoS_LDAP = "C:\\CIC-DDOS-2019\\DrDoS_LDAP.csv"
path_DdoS_NTP = "C:\\CIC-DDOS-2019\\DrDoS_NTP.csv"
path_DdoS_NetBIOS = "C:\\CIC-DDOS-2019\\DrDoS_NetBIOS.csv"
path_DdoS_SNMP = "C:\\CIC-DDOS-2019\\DrDoS_SNMP.csv"
path_DdoS_SSDP = "C:\\CIC-DDOS-2019\\DrDoS_SSDP.csv"
path_DdoS_UDP = "C:\\CIC-DDOS-2019\\DrDoS_UDP.csv"
path_Syn = "C:\\CIC-DDOS-2019\\Syn.csv"
path_TFTP = "C:\\CIC-DDOS-2019\\TFTP.csv"
path_UDPLag = "C:\\CIC-DDOS-2019\\UDPLag.csv"

# # CSV-03-11
# path__LDAP = "../CICDDoS-2019/CSV-03-11/03-11/LDAP.csv"
# path__MSSQL = "../CICDDoS-2019/CSV-03-11/03-11/MSSQL.csv"
# path__NetBIOS = "../CICDDoS-2019/CSV-03-11/03-11/NetBIOS.csv"
# path__Portmap = "../CICDDoS-2019/CSV-03-11/03-11/Portmap.csv"
# path__Syn = "../CICDDoS-2019/CSV-03-11/03-11/Syn.csv"
# path__UDP = "../CICDDoS-2019/CSV-03-11/03-11/UDP.csv"
# path__UDPLag = "../CICDDoS-2019/CSV-03-11/03-11/UDPLag.csv"

paths = [path_DdoS_DNS, path_DdoS_MSSQL, path_DdoS_LDAP, path_DdoS_NTP, path_DdoS_NetBIOS, path_DdoS_SNMP,
         path_DdoS_SSDP, path_DdoS_UDP, path_Syn, path_TFTP, path_UDPLag]
# , path__LDAP, path__MSSQL, path__NetBIOS,
#      path__Portmap, path__Syn, path__UDP, path__UDPLag]

Column / feature names

In [32]:
col_name_consistency = {
    'Flow ID': 'Flow ID',
    'Source IP': 'Source IP',
    'Src IP': 'Source IP',
    'Source Port': 'Source Port',
    'Src Port': 'Source Port',
    'Destination IP': 'Destination IP',
    'Dst IP': 'Destination IP',
    'Destination Port': 'Destination Port',
    'Dst Port': 'Destination Port',
    'Protocol': 'Protocol',
    'Timestamp': 'Timestamp',
    'Flow Duration': 'Flow Duration',
    'Total Fwd Packets': 'Total Fwd Packets',
    'Tot Fwd Pkts': 'Total Fwd Packets',
    'Total Backward Packets': 'Total Backward Packets',
    'Tot Bwd Pkts': 'Total Backward Packets',
    'Total Length of Fwd Packets': 'Fwd Packets Length Total',
    'TotLen Fwd Pkts': 'Fwd Packets Length Total',
    'Total Length of Bwd Packets': 'Bwd Packets Length Total',
    'TotLen Bwd Pkts': 'Bwd Packets Length Total',
    'Fwd Packet Length Max': 'Fwd Packet Length Max',
    'Fwd Pkt Len Max': 'Fwd Packet Length Max',
    'Fwd Packet Length Min': 'Fwd Packet Length Min',
    'Fwd Pkt Len Min': 'Fwd Packet Length Min',
    'Fwd Packet Length Mean': 'Fwd Packet Length Mean',
    'Fwd Pkt Len Mean': 'Fwd Packet Length Mean',
    'Fwd Packet Length Std': 'Fwd Packet Length Std',
    'Fwd Pkt Len Std': 'Fwd Packet Length Std',
    'Bwd Packet Length Max': 'Bwd Packet Length Max',
    'Bwd Pkt Len Max': 'Bwd Packet Length Max',
    'Bwd Packet Length Min': 'Bwd Packet Length Min',
    'Bwd Pkt Len Min': 'Bwd Packet Length Min',
    'Bwd Packet Length Mean': 'Bwd Packet Length Mean',
    'Bwd Pkt Len Mean': 'Bwd Packet Length Mean',
    'Bwd Packet Length Std': 'Bwd Packet Length Std',
    'Bwd Pkt Len Std': 'Bwd Packet Length Std',
    'Flow Bytes/s': 'Flow Bytes/s',
    'Flow Byts/s': 'Flow Bytes/s',
    'Flow Packets/s': 'Flow Packets/s',
    'Flow Pkts/s': 'Flow Packets/s',
    'Flow IAT Mean': 'Flow IAT Mean',
    'Flow IAT Std': 'Flow IAT Std',
    'Flow IAT Max': 'Flow IAT Max',
    'Flow IAT Min': 'Flow IAT Min',
    'Fwd IAT Total': 'Fwd IAT Total',
    'Fwd IAT Tot': 'Fwd IAT Total',
    'Fwd IAT Mean': 'Fwd IAT Mean',
    'Fwd IAT Std': 'Fwd IAT Std',
    'Fwd IAT Max': 'Fwd IAT Max',
    'Fwd IAT Min': 'Fwd IAT Min',
    'Bwd IAT Total': 'Bwd IAT Total',
    'Bwd IAT Tot': 'Bwd IAT Total',
    'Bwd IAT Mean': 'Bwd IAT Mean',
    'Bwd IAT Std': 'Bwd IAT Std',
    'Bwd IAT Max': 'Bwd IAT Max',
    'Bwd IAT Min': 'Bwd IAT Min',
    'Fwd PSH Flags': 'Fwd PSH Flags',
    'Bwd PSH Flags': 'Bwd PSH Flags',
    'Fwd URG Flags': 'Fwd URG Flags',
    'Bwd URG Flags': 'Bwd URG Flags',
    'Fwd Header Length': 'Fwd Header Length',
    'Fwd Header Len': 'Fwd Header Length',
    'Bwd Header Length': 'Bwd Header Length',
    'Bwd Header Len': 'Bwd Header Length',
    'Fwd Packets/s': 'Fwd Packets/s',
    'Fwd Pkts/s': 'Fwd Packets/s',
    'Bwd Packets/s': 'Bwd Packets/s',
    'Bwd Pkts/s': 'Bwd Packets/s',
    'Min Packet Length': 'Packet Length Min',
    'Pkt Len Min': 'Packet Length Min',
    'Max Packet Length': 'Packet Length Max',
    'Pkt Len Max': 'Packet Length Max',
    'Packet Length Mean': 'Packet Length Mean',
    'Pkt Len Mean': 'Packet Length Mean',
    'Packet Length Std': 'Packet Length Std',
    'Pkt Len Std': 'Packet Length Std',
    'Packet Length Variance': 'Packet Length Variance',
    'Pkt Len Var': 'Packet Length Variance',
    'FIN Flag Count': 'FIN Flag Count',
    'FIN Flag Cnt': 'FIN Flag Count',
    'SYN Flag Count': 'SYN Flag Count',
    'SYN Flag Cnt': 'SYN Flag Count',
    'RST Flag Count': 'RST Flag Count',
    'RST Flag Cnt': 'RST Flag Count',
    'PSH Flag Count': 'PSH Flag Count',
    'PSH Flag Cnt': 'PSH Flag Count',
    'ACK Flag Count': 'ACK Flag Count',
    'ACK Flag Cnt': 'ACK Flag Count',
    'URG Flag Count': 'URG Flag Count',
    'URG Flag Cnt': 'URG Flag Count',
    'CWE Flag Count': 'CWE Flag Count',
    'CWE Flag Cnt': 'CWE Flag Count',
    'ECE Flag Count': 'ECE Flag Count',
    'ECE Flag Cnt': 'ECE Flag Count',
    'Down/Up Ratio': 'Down/Up Ratio',
    'Average Packet Size': 'Avg Packet Size',
    'Pkt Size Avg': 'Avg Packet Size',
    'Avg Fwd Segment Size': 'Avg Fwd Segment Size',
    'Fwd Seg Size Avg': 'Avg Fwd Segment Size',
    'Avg Bwd Segment Size': 'Avg Bwd Segment Size',
    'Bwd Seg Size Avg': 'Avg Bwd Segment Size',
    'Fwd Avg Bytes/Bulk': 'Fwd Avg Bytes/Bulk',
    'Fwd Byts/b Avg': 'Fwd Avg Bytes/Bulk',
    'Fwd Avg Packets/Bulk': 'Fwd Avg Packets/Bulk',
    'Fwd Pkts/b Avg': 'Fwd Avg Packets/Bulk',
    'Fwd Avg Bulk Rate': 'Fwd Avg Bulk Rate',
    'Fwd Blk Rate Avg': 'Fwd Avg Bulk Rate',
    'Bwd Avg Bytes/Bulk': 'Bwd Avg Bytes/Bulk',
    'Bwd Byts/b Avg': 'Bwd Avg Bytes/Bulk',
    'Bwd Avg Packets/Bulk': 'Bwd Avg Packets/Bulk',
    'Bwd Pkts/b Avg': 'Bwd Avg Packets/Bulk',
    'Bwd Avg Bulk Rate': 'Bwd Avg Bulk Rate',
    'Bwd Blk Rate Avg': 'Bwd Avg Bulk Rate',
    'Subflow Fwd Packets': 'Subflow Fwd Packets',
    'Subflow Fwd Pkts': 'Subflow Fwd Packets',
    'Subflow Fwd Bytes': 'Subflow Fwd Bytes',
    'Subflow Fwd Byts': 'Subflow Fwd Bytes',
    'Subflow Bwd Packets': 'Subflow Bwd Packets',
    'Subflow Bwd Pkts': 'Subflow Bwd Packets',
    'Subflow Bwd Bytes': 'Subflow Bwd Bytes',
    'Subflow Bwd Byts': 'Subflow Bwd Bytes',
    'Init_Win_bytes_forward': 'Init Fwd Win Bytes',
    'Init Fwd Win Byts': 'Init Fwd Win Bytes',
    'Init_Win_bytes_backward': 'Init Bwd Win Bytes',
    'Init Bwd Win Byts': 'Init Bwd Win Bytes',
    'act_data_pkt_fwd': 'Fwd Act Data Packets',
    'Fwd Act Data Pkts': 'Fwd Act Data Packets',
    'min_seg_size_forward': 'Fwd Seg Size Min',
    'Fwd Seg Size Min': 'Fwd Seg Size Min',
    'Active Mean': 'Active Mean',
    'Active Std': 'Active Std',
    'Active Max': 'Active Max',
    'Active Min': 'Active Min',
    'Idle Mean': 'Idle Mean',
    'Idle Std': 'Idle Std',
    'Idle Max': 'Idle Max',
    'Idle Min': 'Idle Min',
    'Label': 'Label'
}

Following Columns may have little insignificance over model

In [33]:
drop_columns = [  # this list includes all spellings across CIC NIDS datasets
    "Flow ID",
    'Fwd Header Length.1',
    "Source IP",
    "Src IP",
    "Source Port",
    "Src Port",
    "Destination IP",
    "Dst IP",
    "Destination Port",
    "Dst Port",
    "Timestamp",
    "Unnamed: 0",
    "Inbound",
    "SimillarHTTP"  # CIC-DDoS other undocumented columns
]
len(drop_columns)  # src_port,dst_port, src_ip, dst_ip these are duplicate

14

In [34]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
tf.config.experimental.list_physical_devices('GPU')
# import cudf
# dtypes = {'SimillarHTTP': 'object'}
# df_gpu = cudf.read_csv(path_TFTP, blocksize=50e6, low_memory=False, dtype=dtypes)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2997894037423330845
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2966106932
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1670120550598165016
physical_device_desc: "device: 0, name: NVIDIA GeForce 940MX, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [35]:
def readHugeCsvFileAsDataFrame(file_path):
    # start = timer()
    dtypes = {'SimillarHTTP': 'object',
              'Timestamp': 'object',
              'Source IP': 'str',
              'Destination IP': 'str',
              'Flow ID': 'object',
              'Label': 'object',
              }
    for feature in [f'f_{i}' for i in range(82)]:
        dtypes[feature] = "float32"

    dask_df = dd.read_csv(file_path, low_memory=False, dtype=dtypes)  # 50MB chunk-size
    # elapsed_time = timer() - start
    # print("Read csv with dask: ", elapsed_time, "sec")
    return dask_df.compute()

In [36]:
scheduler = Client()
scheduler

C:\Users\Malware\anaconda3\envs\ml_env\lib\site-packages\distributed\node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 12891 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://127.0.0.1:12892 Dashboard: http://127.0.0.1:12891/status,Cluster Workers: 4 Cores: 4 Memory: 8.47 GB


Without Scheduler (see the elapsed time to process)

In [37]:
%%time
# workingDataFrame = readHugeCsvFileAsDataFrame(path_UDPLag)
# workingDataFrame = readHugeCsvFileAsDataFrame(path_Syn)
workingDataFrame = readHugeCsvFileAsDataFrame(path_DdoS_NTP)
# workingDataFrame.describe()
workingDataFrame

Wall time: 1min 19s


,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,0,172.16.0.5-192.168.50.1-60675-80-6,172.16.0.5,60675,192.168.50.1,80,6,2018-12-01 09:17:11.183810,5220876,12,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,205.174.165.72/c.php,1,DrDoS_NTP
1,7,172.16.0.5-192.168.50.1-60676-80-6,172.16.0.5,60676,192.168.50.1,80,6,2018-12-01 09:17:11.205636,12644252,5,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,1,DrDoS_NTP
2,12858,192.168.50.7-65.55.163.78-50458-443-6,65.55.163.78,443,192.168.50.7,50458,6,2018-12-01 09:17:12.634569,3,2,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,1,BENIGN
3,10191,192.168.50.7-65.55.163.78-50465-443-6,65.55.163.78,443,192.168.50.7,50465,6,2018-12-01 09:17:13.458370,3,2,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,1,BENIGN
4,239,192.168.50.253-224.0.0.5-0-0-0,192.168.50.253,0,224.0.0.5,0,0,2018-12-01 09:17:13.470913,114329232,52,...,2.466441,15.0,6.0,9527428.0,248706.681286,9950741.0,9092248.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9321,2414,172.16.0.5-192.168.50.1-634-63125-17,172.16.0.5,634,192.168.50.1,63125,17,2018-12-01 10:51:39.600556,37146,200,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,1,DrDoS_NTP
9322,1506,172.16.0.5-192.168.50.1-634-12431-17,172.16.0.5,634,192.168.50.1,12431,17,2018-12-01 10:51:39.635018,42011,200,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,1,DrDoS_NTP
9323,195,172.16.0.5-192.168.50.1-634-881-17,172.16.0.5,634,192.168.50.1,881,17,2018-12-01 10:51:39.685553,40375,200,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,1,DrDoS_NTP
9324,266,172.16.0.5-192.168.50.1-634-7051-17,172.16.0.5,634,192.168.50.1,7051,17,2018-12-01 10:51:39.721301,59116,200,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,1,DrDoS_NTP


With Scheduler (see the elapsed time to process)

%%time
workingDataFrame = scheduler.submit(readHugeCsvFileAsDataFrame, path_UDPLag)
df = workingDataFrame.result()
df

In [38]:
workingDataFrame.columns

Index(['Unnamed: 0', 'Flow ID', ' Source IP', ' Source Port',
       ' Destination IP', ' Destination Port', ' Protocol', ' Timestamp',
       ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Len

In [39]:
workingDataFrame.isnull().sum()

Unnamed: 0         0
Flow ID            0
 Source IP         0
 Source Port       0
 Destination IP    0
                  ..
 Idle Max          0
 Idle Min          0
SimillarHTTP       0
 Inbound           0
 Label             0
Length: 88, dtype: int64

Dropping Unnecessary Features

In [40]:
workingDataFrame.columns = workingDataFrame.columns.str.strip()  # sometimes there's leading / trailing whitespace
workingDataFrame.drop(columns=drop_columns, inplace=True, errors='ignore')
workingDataFrame.rename(columns=col_name_consistency, inplace=True)

workingDataFrame.head()

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,6,5220876,12,9,1928.0,4290.0,509.0,0.0,160.666667,237.878247,...,32,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,DrDoS_NTP
1,6,12644252,5,2,0.0,0.0,0.0,0.0,0.000000,0.000000,...,32,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,DrDoS_NTP
2,6,3,2,0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,20,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,BENIGN
3,6,3,2,0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,20,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,BENIGN
4,0,114329232,52,0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0,7.416667,2.466441,15.0,6.0,9527428.0,248706.681286,9950741.0,9092248.0,BENIGN


In [41]:
# workingDataFrame.dtypes
workingDataFrame.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1217007 entries, 0 to 9325
Data columns (total 78 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   Protocol                  1217007 non-null  int64  
 1   Flow Duration             1217007 non-null  int64  
 2   Total Fwd Packets         1217007 non-null  int64  
 3   Total Backward Packets    1217007 non-null  int64  
 4   Fwd Packets Length Total  1217007 non-null  float64
 5   Bwd Packets Length Total  1217007 non-null  float64
 6   Fwd Packet Length Max     1217007 non-null  float64
 7   Fwd Packet Length Min     1217007 non-null  float64
 8   Fwd Packet Length Mean    1217007 non-null  float64
 9   Fwd Packet Length Std     1217007 non-null  float64
 10  Bwd Packet Length Max     1217007 non-null  float64
 11  Bwd Packet Length Min     1217007 non-null  float64
 12  Bwd Packet Length Mean    1217007 non-null  float64
 13  Bwd Packet Length Std     1217

Data Cleaning Based on Data Types (DownSizing)

In [42]:
for column in workingDataFrame:
    if workingDataFrame[column].dtype == 'float64':
        workingDataFrame[column] = pd.to_numeric(workingDataFrame[column], downcast='float')
    if workingDataFrame[column].dtype == 'int64':
        workingDataFrame[column] = pd.to_numeric(workingDataFrame[column], downcast='integer')

workingDataFrame.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1217007 entries, 0 to 9325
Data columns (total 78 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   Protocol                  1217007 non-null  int8   
 1   Flow Duration             1217007 non-null  int32  
 2   Total Fwd Packets         1217007 non-null  int16  
 3   Total Backward Packets    1217007 non-null  int16  
 4   Fwd Packets Length Total  1217007 non-null  float32
 5   Bwd Packets Length Total  1217007 non-null  float32
 6   Fwd Packet Length Max     1217007 non-null  float32
 7   Fwd Packet Length Min     1217007 non-null  float32
 8   Fwd Packet Length Mean    1217007 non-null  float32
 9   Fwd Packet Length Std     1217007 non-null  float32
 10  Bwd Packet Length Max     1217007 non-null  float32
 11  Bwd Packet Length Min     1217007 non-null  float32
 12  Bwd Packet Length Mean    1217007 non-null  float32
 13  Bwd Packet Length Std     1217

Removing NaN values

In [43]:
workingDataFrame.isna().any(axis=1).sum()

31

In [44]:
workingDataFrame.replace([np.inf, -np.inf], np.nan, inplace=True)
workingDataFrame.dropna(inplace=True)

After Dropping NaN values, want to see total rows.
That's why, Converting pandas dataFrame into dask's dataFrame

In [45]:
ddf = dd.from_pandas(workingDataFrame, npartitions=10)
ddf.compute()

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,6,5220876,12,9,1928.0,4290.0,509.0,0.0,160.666672,237.878250,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
0,17,372,6,0,2208.0,0.0,440.0,224.0,368.000000,111.541924,...,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
0,17,25893,12,0,5136.0,0.0,440.0,368.0,428.000000,28.025963,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
0,17,3,4,0,1760.0,0.0,440.0,440.0,440.000000,0.000000,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
0,17,3455,136,0,59840.0,0.0,440.0,440.0,440.000000,0.000000,...,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121802,17,6455,20,0,8224.0,0.0,440.0,152.0,411.200012,88.644531,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
121803,17,5591,200,0,88000.0,0.0,440.0,440.0,440.000000,0.000000,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
121804,17,13567,130,0,57200.0,0.0,440.0,440.0,440.000000,0.000000,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP
121805,17,5860,12,0,5280.0,0.0,440.0,440.0,440.000000,0.000000,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_NTP


Dropping Duplicates
There should be no duplicates because they can bias training and can lead to over-optimistic estimates of classification performance during testing.

In [46]:
workingDataFrame.duplicated().sum()

86449

Fully duplicate rows to be removed

In [47]:
workingDataFrame.drop_duplicates(inplace=True)
workingDataFrame.reset_index(inplace=True, drop=True)
workingDataFrame

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,6,5220876,12,9,1928.0,4290.0,509.0,0.0,160.666672,237.878250,...,32,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,DrDoS_NTP
1,6,12644252,5,2,0.0,0.0,0.0,0.0,0.000000,0.000000,...,32,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,DrDoS_NTP
2,6,3,2,0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,20,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,BENIGN
3,0,114329232,52,0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0,7.416667,2.466441,15.0,6.0,9527428.0,248706.687500,9950741.0,9092248.0,BENIGN
4,6,34847273,9,8,8.0,0.0,1.0,0.0,0.888889,0.333333,...,20,33004.000000,32.908966,33030.0,32967.0,10004347.0,8980.692383,10014717.0,9999149.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123507,17,37146,200,0,88000.0,0.0,440.0,440.0,440.000000,0.000000,...,20,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,DrDoS_NTP
1123508,17,42011,200,0,88000.0,0.0,440.0,440.0,440.000000,0.000000,...,20,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,DrDoS_NTP
1123509,17,40375,200,0,88000.0,0.0,440.0,440.0,440.000000,0.000000,...,20,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,DrDoS_NTP
1123510,17,59116,200,0,88000.0,0.0,440.0,440.0,440.000000,0.000000,...,20,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,DrDoS_NTP


After data cleaning, how much storage it holds now

In [48]:
workingDataFrame.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1123512 entries, 0 to 1123511
Data columns (total 78 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   Protocol                  1123512 non-null  int8   
 1   Flow Duration             1123512 non-null  int32  
 2   Total Fwd Packets         1123512 non-null  int16  
 3   Total Backward Packets    1123512 non-null  int16  
 4   Fwd Packets Length Total  1123512 non-null  float32
 5   Bwd Packets Length Total  1123512 non-null  float32
 6   Fwd Packet Length Max     1123512 non-null  float32
 7   Fwd Packet Length Min     1123512 non-null  float32
 8   Fwd Packet Length Mean    1123512 non-null  float32
 9   Fwd Packet Length Std     1123512 non-null  float32
 10  Bwd Packet Length Max     1123512 non-null  float32
 11  Bwd Packet Length Min     1123512 non-null  float32
 12  Bwd Packet Length Mean    1123512 non-null  float32
 13  Bwd Packet Length Std     1

In [49]:
columnList = workingDataFrame.columns.tolist()
# columnList[:-1]  # just for now, Omitting 'label' column

from sklearn.feature_selection import VarianceThreshold

var_thr = VarianceThreshold(threshold=0.1)  #Removing both constant and quasi-constant
var_thr.fit(workingDataFrame[columnList[:-1]])

var_threshold_bool_list = var_thr.get_support()
var_threshold_bool_list_after_label_added = np.append(var_threshold_bool_list,True)
var_threshold_bool_list_after_label_added

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False, False, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True, False, False, False, False, False, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [50]:
concol = [column for column in workingDataFrame.columns
          if column not in workingDataFrame.columns[var_threshold_bool_list_after_label_added]]

for omittedFeatures in concol:
    print(omittedFeatures)

workingDataFrame.drop(columns=concol, inplace=True, errors='ignore')
workingDataFrame

Fwd PSH Flags
Bwd PSH Flags
Fwd URG Flags
Bwd URG Flags
FIN Flag Count
SYN Flag Count
RST Flag Count
PSH Flag Count
ACK Flag Count
URG Flag Count
CWE Flag Count
ECE Flag Count
Down/Up Ratio
Fwd Avg Bytes/Bulk
Fwd Avg Packets/Bulk
Fwd Avg Bulk Rate
Bwd Avg Bytes/Bulk
Bwd Avg Packets/Bulk
Bwd Avg Bulk Rate


,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,6,5220876,12,9,1928.0,4290.0,509.0,0.0,160.666672,237.878250,...,32,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,DrDoS_NTP
1,6,12644252,5,2,0.0,0.0,0.0,0.0,0.000000,0.000000,...,32,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,DrDoS_NTP
2,6,3,2,0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,20,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,BENIGN
3,0,114329232,52,0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0,7.416667,2.466441,15.0,6.0,9527428.0,248706.687500,9950741.0,9092248.0,BENIGN
4,6,34847273,9,8,8.0,0.0,1.0,0.0,0.888889,0.333333,...,20,33004.000000,32.908966,33030.0,32967.0,10004347.0,8980.692383,10014717.0,9999149.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123507,17,37146,200,0,88000.0,0.0,440.0,440.0,440.000000,0.000000,...,20,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,DrDoS_NTP
1123508,17,42011,200,0,88000.0,0.0,440.0,440.0,440.000000,0.000000,...,20,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,DrDoS_NTP
1123509,17,40375,200,0,88000.0,0.0,440.0,440.0,440.000000,0.000000,...,20,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,DrDoS_NTP
1123510,17,59116,200,0,88000.0,0.0,440.0,440.0,440.000000,0.000000,...,20,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,DrDoS_NTP


Now arising problem is that, row number has been significantly reduced but feature numbers are still 78!!
So, need feature engineering here

In [51]:

def featueEngineeringBasedOnZero(dataFrameArg, thresholdPercentage, showPercentage):
    totalCols = dataFrameArg.shape[1]
    totalRows = len(dataFrameArg)
    unNecessaryFeatureCount = 0
    unNecessaryFeatureNames = []

    for column in dataFrameArg:
        zerosInCol = (dataFrameArg[column] == 0).sum()
        if zerosInCol != 0:
            percentageOfZerosInRow = ((zerosInCol * 100) / totalRows)

            if showPercentage:
                print(column, "\t\t-\t\t", zerosInCol, "\t\t-\t\t", percentageOfZerosInRow)

            if percentageOfZerosInRow > thresholdPercentage:
                unNecessaryFeatureNames.append(column)
                unNecessaryFeatureCount = unNecessaryFeatureCount + 1

    print("\nTotal features having more than ", thresholdPercentage, "% zero are - ", unNecessaryFeatureCount,
          "out of ",
          totalCols)
    return unNecessaryFeatureNames

Identifying those features containing 99% zeroes

In [52]:
featureContainingAlmostZero = featueEngineeringBasedOnZero(dataFrameArg=workingDataFrame, thresholdPercentage=99,
                                                           showPercentage=False)
featureContainingAlmostZero


Total features having more than  99 % zero are -  16 out of  59


['Bwd Packets Length Total',
 'Bwd Packet Length Max',
 'Bwd Packet Length Min',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Std',
 'Bwd IAT Std',
 'Avg Bwd Segment Size',
 'Subflow Bwd Bytes',
 'Active Mean',
 'Active Std',
 'Active Max',
 'Active Min',
 'Idle Mean',
 'Idle Std',
 'Idle Max',
 'Idle Min']

Omitting above features containing 99% zeroes

In [53]:
workingDataFrame.drop(columns=featureContainingAlmostZero, inplace=True, errors='ignore')
workingDataFrame.rename(columns=col_name_consistency, inplace=True)
workingDataFrame.reset_index(inplace=True, drop=True)
workingDataFrame

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Flow Bytes/s,...,Avg Packet Size,Avg Fwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Init Fwd Win Bytes,Init Bwd Win Bytes,Fwd Act Data Packets,Fwd Seg Size Min,Label
0,6,5220876,12,9,1928.0,509.0,0.0,160.666672,237.878250,1.190988e+03,...,296.095245,160.666672,12,1928,9,65535,243,4,32,DrDoS_NTP
1,6,12644252,5,2,0.0,0.0,0.0,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,5,0,2,65535,28960,0,32,DrDoS_NTP
2,6,3,2,0,0.0,0.0,0.0,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,2,0,0,0,-1,0,20,BENIGN
3,0,114329232,52,0,0.0,0.0,0.0,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,52,0,0,-1,-1,0,0,BENIGN
4,6,34847273,9,8,8.0,1.0,0.0,0.888889,0.333333,2.295732e-01,...,0.529412,0.888889,9,8,8,258,254,7,20,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123507,17,37146,200,0,88000.0,440.0,440.0,440.000000,0.000000,2.369030e+06,...,442.200012,440.000000,200,88000,0,-1,-1,199,20,DrDoS_NTP
1123508,17,42011,200,0,88000.0,440.0,440.0,440.000000,0.000000,2.094690e+06,...,442.200012,440.000000,200,88000,0,-1,-1,199,20,DrDoS_NTP
1123509,17,40375,200,0,88000.0,440.0,440.0,440.000000,0.000000,2.179566e+06,...,442.200012,440.000000,200,88000,0,-1,-1,199,20,DrDoS_NTP
1123510,17,59116,200,0,88000.0,440.0,440.0,440.000000,0.000000,1.488599e+06,...,442.200012,440.000000,200,88000,0,-1,-1,199,20,DrDoS_NTP


Now, To see the number of unique values in each column

In [54]:
workingDataFrame.nunique(axis=0)

Protocol                          3
Flow Duration                 63665
Total Fwd Packets               294
Total Backward Packets          191
Fwd Packets Length Total       2664
Fwd Packet Length Max           896
Fwd Packet Length Min            83
Fwd Packet Length Mean         2426
Fwd Packet Length Std          3098
Flow Bytes/s                 451464
Flow Packets/s               292896
Flow IAT Mean                350429
Flow IAT Std                1045430
Flow IAT Max                  36346
Flow IAT Min                   1033
Fwd IAT Total                 61340
Fwd IAT Mean                 346527
Fwd IAT Std                 1041211
Fwd IAT Max                   33343
Fwd IAT Min                     244
Bwd IAT Total                  4173
Bwd IAT Mean                   4229
Bwd IAT Max                    3914
Bwd IAT Min                      62
Fwd Header Length              6084
Bwd Header Length               365
Fwd Packets/s                292986
Bwd Packets/s               

In [55]:
workingDataFrame

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Flow Bytes/s,...,Avg Packet Size,Avg Fwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Init Fwd Win Bytes,Init Bwd Win Bytes,Fwd Act Data Packets,Fwd Seg Size Min,Label
0,6,5220876,12,9,1928.0,509.0,0.0,160.666672,237.878250,1.190988e+03,...,296.095245,160.666672,12,1928,9,65535,243,4,32,DrDoS_NTP
1,6,12644252,5,2,0.0,0.0,0.0,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,5,0,2,65535,28960,0,32,DrDoS_NTP
2,6,3,2,0,0.0,0.0,0.0,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,2,0,0,0,-1,0,20,BENIGN
3,0,114329232,52,0,0.0,0.0,0.0,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,52,0,0,-1,-1,0,0,BENIGN
4,6,34847273,9,8,8.0,1.0,0.0,0.888889,0.333333,2.295732e-01,...,0.529412,0.888889,9,8,8,258,254,7,20,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123507,17,37146,200,0,88000.0,440.0,440.0,440.000000,0.000000,2.369030e+06,...,442.200012,440.000000,200,88000,0,-1,-1,199,20,DrDoS_NTP
1123508,17,42011,200,0,88000.0,440.0,440.0,440.000000,0.000000,2.094690e+06,...,442.200012,440.000000,200,88000,0,-1,-1,199,20,DrDoS_NTP
1123509,17,40375,200,0,88000.0,440.0,440.0,440.000000,0.000000,2.179566e+06,...,442.200012,440.000000,200,88000,0,-1,-1,199,20,DrDoS_NTP
1123510,17,59116,200,0,88000.0,440.0,440.0,440.000000,0.000000,1.488599e+06,...,442.200012,440.000000,200,88000,0,-1,-1,199,20,DrDoS_NTP


Saving New DataFrame as csv file to new location

In [56]:
def dataCleaningResultToAnotherCSV(dataFrameArg, dirPath, file_name):
    dataFrameArg.to_csv(dirPath + file_name)

In [57]:
newCsvPath = "C:\\CIC-DDOS-2019\\AfterDataCleaning\\"
# newFileName = "UDPLag.csv"
# newFileName = "Syn.csv"
newFileName = "DrDoS_NTP.csv"
dataCleaningResultToAnotherCSV(dataFrameArg=workingDataFrame, dirPath=newCsvPath, file_name=newFileName)

In [58]:
gc.collect()

25